# Evaluation - Notebook

Within this document, we carry out an assessment of the implemented models. Furthermore, hyperparameter optimization is executed for each model wherever feasible. The following models are subjected to evaluation:

1. Baseline
2. Moving Average
3. Various variants of linear regression
4. Neural Network
5. XGBoost
6. LSTM

For a more detailed description of the models, please refeir to their corresponding section.

In [1]:
# TODO: look into pipeline framework for hyperparameter tuning
# ###: 1. look into warning for large window sizes
# ###: 2. polynomial features for linear regression 
# ###: 3. Different Features (difficult) -> solution might be time-series split
# TODO: do analysis also on brazilian data
# TODO: Rewrite loading.py file
# TODO: try the transfer learning approach
# TODO: Plotting
# TODO: implement more models
# TODO: implement command line interface - optional 
# TODO: implement logging - optional
# TODO: Documentation 
# TODO: Hand-in

In [2]:
### imports
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit

import sys

sys.path.append("..")
sys.path.append("../src")

# plotting
import src.DataHandling.visualization as vis

# models
from src.Models.ma import MovingAverage
from src.Models.lr import Regression

# Transformers
from src.DataHandling.preprocessing import (
    DataCleaner,
    SupervisedTransformer,
    train_test_split,
)

# hyperparameter tuning
from src.Models.selection import GridSearch

In [3]:
# load cleaned data
turbine_brit = pd.read_csv(f"../data/cleaned/turbine_brit_{2}.csv")
turbine_braz = pd.read_csv(f"../data/cleaned/turbine_braz_{1}.csv")

# use date column as index and convert to datetime
turbine_brit["Date"] = pd.to_datetime(turbine_brit["Date"])
turbine_brit.set_index("Date", inplace=True)

turbine_braz["Date"] = pd.to_datetime(turbine_braz["Date"])
turbine_braz.set_index("Date", inplace=True)

DATA = {"British": turbine_brit, "Brazilian": turbine_braz}

,Power (kW),Wind speed (m/s)
Date,,
2016-01-21 17:40:00,90.379997,7.97
2016-01-21 17:50:00,1.330000,8.20
2016-01-21 18:00:00,240.740005,9.04
2016-01-21 18:10:00,-2.770000,9.26
2016-01-21 18:20:00,-2.350000,9.50
...,...,...
2021-06-30 23:10:00,113.809998,3.85
2021-06-30 23:20:00,128.860001,4.06
2021-06-30 23:30:00,112.220001,4.00


In [8]:
# load data
data = 

cleaner = DataCleaner(
    features=["Power (kW)", "Wind speed (m/s)"]
)
turbine_brit.equals(cleaner.transform(turbine_brit))

True

For understandibility.... models do transformation to a supervised learning problem inherently...

In [ ]:
benchmarks = pd.read_csv("../results_wind.csv")
test_start = benchmarks["test_start"][0]
test_end = benchmarks["test_end"][0]

X_train, y_train, X_test, y_test = train_test_split(
    test_start=test_start, test_end=test_end, df=turbine_brit, target_var="Power (kW)"
)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

Before, we begin blablabla ... there are shared parameters across models like window_size etc.

In [ ]:
# set up parameters that are the same for all models
param_grid = {
    "st__window_size": [1, 5, 10],
    "st__horizon": [1, 6, 144],
}

## 1. Moving Average

**Explanation of the Model:** 

In contrast to the Baseline model, the Moving Average model offers the flexibility of selecting a window size. This window size determines the number of preceding time steps taken into account for generating forecasts. Furthermore, there is an option to specify a discount factor (default = 1), which regulates the extent of reduction in influence for more distant past time steps. This strategy is similar to how a discount factor is used in calculating future rewards within the framework of reinforcement learning. Note that when a window size of one is chosen, the Moving Average model is identical with the Baseline model.

In [ ]:
ma_pipe = Pipeline(
    [
        ("st", SupervisedTransformer()),
        ("Model", MovingAverage()),
    ]
)

param_grid["Model__discount"] = np.linspace(0.5, 1., 3)

grid_search_ma = GridSearch(ma_pipe, param_grid)
grid_search_ma.fit(X_train, y_train, X_test, y_test)

In [ ]:
# delete key of param_grid that is specific to the model
del param_grid["Model__discount"]
param_grid

## 2. Different kinds of Linear Regression

**Explanation of the Models:**


In [ ]:
lr_pipe = Pipeline(
    [
        ("st", SupervisedTransformer()),
        ("Model", Regression()),
    ]
)

param_grid["Model__model"] = ["linear", "ridge"]
param_grid["Model__alpha"] = np.logspace(-3, 1, 5)

In [ ]:
param_grid["Model__alpha"]

In [ ]:
grid_search_lr = GridSearch(lr_pipe, param_grid)
grid_search_lr.fit(X_train, y_train, X_test, y_test)
grid_search_lr.best_params

In [ ]:
# delete key of param_grid that is specific to the model
del param_grid["Model__model"]
del param_grid["Model__alpha"]
param_grid